In [21]:
!pip install datasets
!pip install peft transformers

In [22]:
from datasets import load_dataset
import pandas as pd
import re

In [33]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas and keep only useful columns
df_train = ds["train"].to_pandas()[["question","interview_answer",
                                   "label"]]

In [24]:
df_train

,question,interview_answer,label
0,How would you respond to the accusation that t...,"\nThe President. Well, look, first of all, the...",Explicit
1,Do you think President Xi is being sincere abo...,"\nThe President. Well, look, first of all, the...",General
2,1. Q1: Do you believe the country's slowdown a...,"\nThe President. Look, I think China has a dif...",Partial/half-answer
3,2. Q2: Are you worried about the meeting betwe...,"\nThe President. Look, I think China has a dif...",Dodging
4,Is the President's engagement with Asian coun...,"\nThe President. Well, I hope I get to see Mr....",Explicit
...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,\nThe President. That's a tricky little questi...,Deflection
3444,Inquiry about the belief regarding the abilit...,"\nThe President. Mike, I believe Iraq will be ...",Explicit
3445,2. Are you resentful that some Republican cand...,"\nThe President. You know, no, I'm not resentf...",Explicit
3446,1. If you really didn't think that Republicans...,"\nThe President. You know, no, I'm not resentf...",Deflection


In [20]:
# Regex explanation:

# ^ matches the start of the string
# (\d+(?:\. ?|\. Q\d+: ?)|Part \d+: ?) is a capturing group that matches the inconsistencies:
#     \d+ matches one or more digits
#     (?:\. ?|\. Q\d+: ?) is a non-capturing group that matches either:
#         \. matches a period
#         ? matches an optional space
#         | is a logical OR operator
#         \. Q\d+: ? matches a period, "Q", digits, colon, and an optional space
#     | is a logical OR operator
#     Part \d+: ? matches "Part", digits, colon, and an optional space
#     ? at the end makes the whole group optional, in case the text doesn't have any inconsistencies

In [34]:
# Remove indexing from questions
index_pattern = r'^(\d+(?:\. ?|\. Q\d+: ?)|Part \d+: ?)'
df_train['question'] = df_train['question'].str.replace(index_pattern, '', regex=True)

# Remove quotes
df_train['question'] = df_train['question'].str.replace('"', '')

In [40]:
df_train

,question,interview_answer,label
0,How would you respond to the accusation that t...,"\nThe President. Well, look, first of all, the...",Explicit
1,Do you think President Xi is being sincere abo...,"\nThe President. Well, look, first of all, the...",General
2,Q1: Do you believe the country's slowdown and ...,"\nThe President. Look, I think China has a dif...",Partial/half-answer
3,Q2: Are you worried about the meeting between ...,"\nThe President. Look, I think China has a dif...",Dodging
4,Is the President's engagement with Asian coun...,"\nThe President. Well, I hope I get to see Mr....",Explicit
...,...,...,...
3443,Why shouldn't Americans give Democrats a chan...,\nThe President. That's a tricky little questi...,Deflection
3444,Inquiry about the belief regarding the abilit...,"\nThe President. Mike, I believe Iraq will be ...",Explicit
3445,Are you resentful that some Republican candida...,"\nThe President. You know, no, I'm not resentf...",Explicit
3446,If you really didn't think that Republicans wo...,"\nThe President. You know, no, I'm not resentf...",Deflection
